# Model Capacity Effects: Transformers vs Tree-Based Models

## Objective
Understand how **increasing model capacity** affects bias and variance differently in:
1. **Tree-based models** (Decision Trees, Random Forests, Gradient Boosting)
2. **Transformer-based models** (Attention mechanisms, deep architectures)

## Key Question
Why do these architectures respond differently to capacity increases?

## Hypothesis
- **Trees**: Sharp transition from underfitting to overfitting as capacity increases (depth/trees)
- **Transformers**: More gradual scaling, can benefit from overparameterization due to implicit regularization

## What We'll Explore
1. Generate synthetic data with complex feature interactions
2. Train models with varying capacity levels
3. Measure bias, variance, and generalization at each capacity
4. Visualize decision boundaries
5. Explain the fundamental architectural differences

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")

## Part 1: Synthetic Dataset Design

We need a dataset that:
1. Has **complex feature interactions** (benefits transformers' attention)
2. Has **non-linear patterns** (challenges simple trees)
3. Is **high-dimensional** (shows capacity scaling effects)

### Dataset Characteristics:
- **Task**: Multi-class classification (3 classes)
- **Features**: 20 features with various interaction patterns
- **Interactions**: 
  - Multiplicative interactions (feature_i × feature_j)
  - Non-linear transformations
  - Long-range dependencies (simulates sequential context)
- **Sample size**: Moderate (to see overfitting effects)

This mimics scenarios where:
- Trees need depth to capture interactions
- Transformers can use attention to capture relationships

In [ ]:
def generate_complex_dataset(n_samples=1000, n_features=20, noise_level=0.1, random_state=42):
    """
    Generate synthetic dataset with complex feature interactions.
    Designed to test both tree-based and attention-based models.
    """
    np.random.seed(random_state)
    
    # Generate base features
    X = np.random.randn(n_samples, n_features)
    
    # Create complex decision boundaries using multiple interaction patterns
    
    # Pattern 1: Multiplicative interactions (features 0-5)
    interaction_1 = (X[:, 0] * X[:, 1]) + (X[:, 2] * X[:, 3])
    
    # Pattern 2: Non-linear transformations (features 6-10)
    interaction_2 = np.sin(X[:, 6]) * np.cos(X[:, 7]) + np.tanh(X[:, 8] * X[:, 9])
    
    # Pattern 3: Long-range dependencies (features 11-19)
    # Simulate sequential context: each feature depends on previous ones
    interaction_3 = np.sum([X[:, i] * X[:, i+1] * (i/n_features) 
                           for i in range(11, 18)], axis=0)
    
    # Pattern 4: XOR-like pattern (features 15-16)
    interaction_4 = ((X[:, 15] > 0) ^ (X[:, 16] > 0)).astype(float) * 2 - 1
    
    # Combine patterns to create target
    decision_function = (
        0.4 * interaction_1 + 
        0.3 * interaction_2 + 
        0.2 * interaction_3 + 
        0.1 * interaction_4
    )
    
    # Add noise
    decision_function += np.random.normal(0, noise_level, n_samples)
    
    # Create 3 classes based on quantiles
    quantiles = np.percentile(decision_function, [33, 67])
    y = np.zeros(n_samples, dtype=int)
    y[decision_function > quantiles[1]] = 2
    y[(decision_function > quantiles[0]) & (decision_function <= quantiles[1])] = 1
    
    return X, y, decision_function

# Generate datasets
print("Generating datasets...")
X_full, y_full, decision_values = generate_complex_dataset(
    n_samples=2000, n_features=20, noise_level=0.15
)

# Split into train, validation, and test
X_temp, X_test, y_temp, y_test = train_test_split(
    X_full, y_full, test_size=0.3, random_state=42, stratify=y_full
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Number of features: {X_train.shape[1]}")
print(f"Number of classes: {len(np.unique(y_train))}")
print(f"\nClass distribution (train): {np.bincount(y_train)}")

In [ ]:
# Visualize first two principal components and feature correlations
from sklearn.decomposition import PCA

# PCA for visualization
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Data in 2D PCA space
scatter = axes[0].scatter(X_train_pca[:, 0], X_train_pca[:, 1], 
                          c=y_train, cmap='viridis', 
                          s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('First Principal Component', fontsize=12)
axes[0].set_ylabel('Second Principal Component', fontsize=12)
axes[0].set_title('Training Data in PCA Space\n(Complex interactions not fully visible in 2D)', 
                  fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0], label='Class')

# Plot 2: Feature correlation matrix
correlation_matrix = np.corrcoef(X_train.T)
im = axes[1].imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
axes[1].set_xlabel('Feature Index', fontsize=12)
axes[1].set_ylabel('Feature Index', fontsize=12)
axes[1].set_title('Feature Correlation Matrix\n(Shows feature interaction structure)', 
                  fontsize=12, fontweight='bold')
plt.colorbar(im, ax=axes[1], label='Correlation')

plt.tight_layout()
plt.show()

print("\nDataset Complexity Indicators:")
print(f"Explained variance by first 2 PCs: {sum(pca.explained_variance_ratio_):.2%}")
print(f"Average absolute correlation: {np.mean(np.abs(correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)])):.3f}")

## Part 2: Tree-Based Models

We'll test three tree-based architectures with varying capacity:

### 1. **Decision Tree** (Single Tree)
- **Capacity control**: `max_depth` (2, 5, 10, 15, 20, None)
- **Mechanism**: Recursive axis-aligned splits
- **Interaction modeling**: Explicit through depth

### 2. **Random Forest** (Ensemble of Trees)
- **Capacity control**: `n_estimators` and `max_depth`
- **Mechanism**: Bootstrap aggregating (bagging)
- **Variance reduction**: Through averaging

### 3. **Gradient Boosting** (Sequential Ensemble)
- **Capacity control**: `n_estimators` and `max_depth`
- **Mechanism**: Additive model building
- **Bias reduction**: Sequential error correction

### Key Properties of Trees:
- **Local decisions**: Each split affects only a subset of data
- **Greedy learning**: No backpropagation or global optimization
- **Hard partitions**: Axis-aligned rectangular regions
- **Exponential capacity**: Depth d → 2^d leaf nodes

In [ ]:
def train_tree_models_varying_capacity():
    """Train tree-based models with different capacity levels"""
    
    # Decision Tree: vary depth
    dt_capacities = [2, 5, 10, 15, 20, None]
    dt_results = []
    
    print("Training Decision Trees...")
    for depth in dt_capacities:
        model = DecisionTreeClassifier(max_depth=depth, random_state=42)
        model.fit(X_train, y_train)
        
        train_acc = accuracy_score(y_train, model.predict(X_train))
        val_acc = accuracy_score(y_val, model.predict(X_val))
        test_acc = accuracy_score(y_test, model.predict(X_test))
        
        # Calculate probabilities for loss
        train_proba = model.predict_proba(X_train)
        val_proba = model.predict_proba(X_val)
        
        train_loss = log_loss(y_train, train_proba)
        val_loss = log_loss(y_val, val_proba)
        
        dt_results.append({
            'capacity': depth if depth else 'unlimited',
            'depth': depth if depth else 20,  # for plotting
            'model': model,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'test_acc': test_acc,
            'train_loss': train_loss,
            'val_loss': val_loss
        })
        
        print(f"  Depth {depth}: Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")
    
    # Random Forest: vary number of estimators (fixed depth)
    rf_capacities = [1, 5, 10, 25, 50, 100, 200]
    rf_results = []
    
    print("\nTraining Random Forests...")
    for n_est in rf_capacities:
        model = RandomForestClassifier(
            n_estimators=n_est, max_depth=10, random_state=42, n_jobs=-1
        )
        model.fit(X_train, y_train)
        
        train_acc = accuracy_score(y_train, model.predict(X_train))
        val_acc = accuracy_score(y_val, model.predict(X_val))
        test_acc = accuracy_score(y_test, model.predict(X_test))
        
        train_proba = model.predict_proba(X_train)
        val_proba = model.predict_proba(X_val)
        
        train_loss = log_loss(y_train, train_proba)
        val_loss = log_loss(y_val, val_proba)
        
        rf_results.append({
            'capacity': n_est,
            'model': model,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'test_acc': test_acc,
            'train_loss': train_loss,
            'val_loss': val_loss
        })
        
        print(f"  n_estimators {n_est}: Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")
    
    # Gradient Boosting: vary number of estimators
    gb_capacities = [1, 5, 10, 25, 50, 100, 200]
    gb_results = []
    
    print("\nTraining Gradient Boosting...")
    for n_est in gb_capacities:
        model = GradientBoostingClassifier(
            n_estimators=n_est, max_depth=5, learning_rate=0.1, random_state=42
        )
        model.fit(X_train, y_train)
        
        train_acc = accuracy_score(y_train, model.predict(X_train))
        val_acc = accuracy_score(y_val, model.predict(X_val))
        test_acc = accuracy_score(y_test, model.predict(X_test))
        
        train_proba = model.predict_proba(X_train)
        val_proba = model.predict_proba(X_val)
        
        train_loss = log_loss(y_train, train_proba)
        val_loss = log_loss(y_val, val_proba)
        
        gb_results.append({
            'capacity': n_est,
            'model': model,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'test_acc': test_acc,
            'train_loss': train_loss,
            'val_loss': val_loss
        })
        
        print(f"  n_estimators {n_est}: Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")
    
    return dt_results, rf_results, gb_results

# Train all tree models
dt_results, rf_results, gb_results = train_tree_models_varying_capacity()
print("\n✓ Tree-based models trained")

## Part 3: Transformer-Based Models

We'll implement transformer architectures for tabular data with varying capacity:

### Transformer Components:
1. **Input Embedding**: Project features to hidden dimension
2. **Multi-Head Self-Attention**: Learn feature interactions
3. **Feed-Forward Networks**: Non-linear transformations
4. **Layer Normalization**: Stabilize training
5. **Classification Head**: Final prediction layer

### Capacity Control:
- **Hidden dimension** (d_model): 32, 64, 128, 256
- **Number of layers**: 1, 2, 4, 6
- **Number of attention heads**: 2, 4, 8

### Key Properties of Transformers:
- **Global context**: Attention sees all features simultaneously
- **Soft interactions**: Learned attention weights (not hard splits)
- **Gradient-based**: Backpropagation through entire network
- **Implicit regularization**: Overparameterization can help generalization
- **Polynomial capacity**: Layers × Hidden_dim (more gradual than trees)

In [ ]:
class TabularTransformer(nn.Module):
    """Transformer architecture for tabular data"""
    
    def __init__(self, input_dim, d_model, nhead, num_layers, num_classes, dropout=0.1):
        super(TabularTransformer, self).__init__()
        
        self.input_dim = input_dim
        self.d_model = d_model
        
        # Input projection
        self.input_projection = nn.Linear(input_dim, d_model)
        
        # Positional encoding (for feature position awareness)
        self.pos_embedding = nn.Parameter(torch.randn(1, input_dim, d_model))
        
        # Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Classification head
        self.layer_norm = nn.LayerNorm(d_model)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )
        
    def forward(self, x):
        # x shape: (batch_size, input_dim)
        batch_size = x.shape[0]
        
        # Expand features for attention: (batch, input_dim) -> (batch, input_dim, 1)
        x = x.unsqueeze(-1)  # (batch, input_dim, 1)
        
        # Project to d_model
        x = x.expand(-1, -1, self.d_model)  # (batch, input_dim, d_model)
        x = x * self.input_projection.weight.T + self.input_projection.bias
        
        # Add positional encoding
        x = x + self.pos_embedding
        
        # Transformer encoding
        x = self.transformer_encoder(x)  # (batch, input_dim, d_model)
        
        # Global average pooling
        x = torch.mean(x, dim=1)  # (batch, d_model)
        
        # Classification
        x = self.layer_norm(x)
        x = self.classifier(x)
        
        return x

# Test the architecture
test_model = TabularTransformer(
    input_dim=20, d_model=64, nhead=4, num_layers=2, num_classes=3
)
test_input = torch.randn(10, 20)
test_output = test_model(test_input)
print(f"✓ Transformer architecture working: input {test_input.shape} -> output {test_output.shape}")

# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Test model parameters: {count_parameters(test_model):,}")

In [ ]:
def train_transformer(model, train_loader, val_loader, epochs=50, lr=0.001, device='cpu'):
    """Train a transformer model"""
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_y.size(0)
            train_correct += (predicted == batch_y).sum().item()
        
        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += batch_y.size(0)
                val_correct += (predicted == batch_y).sum().item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        train_acc = train_correct / train_total
        val_acc = val_correct / val_total
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        scheduler.step(val_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"  Epoch {epoch+1}/{epochs}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, "
                  f"Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")
    
    return model, train_losses, val_losses, train_accs, val_accs

def train_transformers_varying_capacity():
    """Train transformers with different capacity levels"""
    
    # Prepare data loaders
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.LongTensor(y_val)
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.LongTensor(y_test)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    # Vary hidden dimension (d_model)
    transformer_configs = [
        {'d_model': 32, 'nhead': 2, 'num_layers': 2, 'name': 'Small (d=32, L=2)'},
        {'d_model': 64, 'nhead': 4, 'num_layers': 2, 'name': 'Medium (d=64, L=2)'},
        {'d_model': 128, 'nhead': 4, 'num_layers': 2, 'name': 'Large (d=128, L=2)'},
        {'d_model': 64, 'nhead': 4, 'num_layers': 1, 'name': 'Shallow (d=64, L=1)'},
        {'d_model': 64, 'nhead': 4, 'num_layers': 4, 'name': 'Deep (d=64, L=4)'},
        {'d_model': 256, 'nhead': 8, 'num_layers': 4, 'name': 'Very Large (d=256, L=4)'},
    ]
    
    transformer_results = []
    
    print("\nTraining Transformers...")
    for config in transformer_configs:
        print(f"\n{config['name']}:")
        
        model = TabularTransformer(
            input_dim=X_train.shape[1],
            d_model=config['d_model'],
            nhead=config['nhead'],
            num_layers=config['num_layers'],
            num_classes=len(np.unique(y_train)),
            dropout=0.1
        )
        
        n_params = count_parameters(model)
        print(f"  Parameters: {n_params:,}")
        
        # Train
        model, train_losses, val_losses, train_accs, val_accs = train_transformer(
            model, train_loader, val_loader, epochs=50, lr=0.001, device=device
        )
        
        # Evaluate on test set
        model.eval()
        test_correct = 0
        test_total = 0
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                _, predicted = torch.max(outputs.data, 1)
                test_total += batch_y.size(0)
                test_correct += (predicted == batch_y).sum().item()
        
        test_acc = test_correct / test_total
        
        transformer_results.append({
            'name': config['name'],
            'config': config,
            'n_params': n_params,
            'model': model,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accs': train_accs,
            'val_accs': val_accs,
            'final_train_acc': train_accs[-1],
            'final_val_acc': val_accs[-1],
            'test_acc': test_acc
        })
        
        print(f"  Final - Train Acc={train_accs[-1]:.3f}, Val Acc={val_accs[-1]:.3f}, Test Acc={test_acc:.3f}")
    
    return transformer_results

# Train all transformer models
transformer_results = train_transformers_varying_capacity()
print("\n✓ Transformer models trained")

## Part 4: Comparing Capacity Scaling

Now let's visualize how bias and variance change with capacity for both model families.

### What to Expect:

**Tree-Based Models:**
- Sharp transition from high bias to high variance
- Clear "sweet spot" in capacity
- Overfitting happens suddenly with too much depth
- Ensemble methods smooth this transition

**Transformers:**
- More gradual capacity scaling
- Can benefit from overparameterization
- Implicit regularization through optimization
- Performance plateau rather than sharp overfitting

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Decision Trees
ax = axes[0, 0]
depths = [r['depth'] for r in dt_results]
train_accs = [r['train_acc'] for r in dt_results]
val_accs = [r['val_acc'] for r in dt_results]
test_accs = [r['test_acc'] for r in dt_results]

ax.plot(depths, train_accs, 'o-', linewidth=2, markersize=8, label='Train', color='blue')
ax.plot(depths, val_accs, 's-', linewidth=2, markersize=8, label='Validation', color='orange')
ax.plot(depths, test_accs, '^-', linewidth=2, markersize=8, label='Test', color='green')
ax.set_xlabel('Max Depth', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Decision Tree: Capacity vs Performance', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xticks(depths)

# Add overfitting annotation
max_depth_idx = np.argmax(val_accs)
ax.axvline(depths[max_depth_idx], color='red', linestyle='--', alpha=0.5, linewidth=2)
ax.text(depths[max_depth_idx], 0.5, 'Optimal\nCapacity', 
        rotation=90, verticalalignment='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

# Random Forest
ax = axes[0, 1]
n_ests = [r['capacity'] for r in rf_results]
train_accs = [r['train_acc'] for r in rf_results]
val_accs = [r['val_acc'] for r in rf_results]
test_accs = [r['test_acc'] for r in rf_results]

ax.plot(n_ests, train_accs, 'o-', linewidth=2, markersize=8, label='Train', color='blue')
ax.plot(n_ests, val_accs, 's-', linewidth=2, markersize=8, label='Validation', color='orange')
ax.plot(n_ests, test_accs, '^-', linewidth=2, markersize=8, label='Test', color='green')
ax.set_xlabel('Number of Trees', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Random Forest: Capacity vs Performance', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

# Gradient Boosting
ax = axes[0, 2]
n_ests = [r['capacity'] for r in gb_results]
train_accs = [r['train_acc'] for r in gb_results]
val_accs = [r['val_acc'] for r in gb_results]
test_accs = [r['test_acc'] for r in gb_results]

ax.plot(n_ests, train_accs, 'o-', linewidth=2, markersize=8, label='Train', color='blue')
ax.plot(n_ests, val_accs, 's-', linewidth=2, markersize=8, label='Validation', color='orange')
ax.plot(n_ests, test_accs, '^-', linewidth=2, markersize=8, label='Test', color='green')
ax.set_xlabel('Number of Boosting Rounds', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Gradient Boosting: Capacity vs Performance', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

# Generalization gap (overfitting indicator)
ax = axes[1, 0]
gaps_dt = [train - val for train, val in zip([r['train_acc'] for r in dt_results], 
                                               [r['val_acc'] for r in dt_results])]
ax.plot(depths, gaps_dt, 'o-', linewidth=2.5, markersize=10, color='red', label='Decision Tree')
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.fill_between(depths, 0, gaps_dt, alpha=0.3, color='red')
ax.set_xlabel('Max Depth', fontsize=12)
ax.set_ylabel('Train Acc - Val Acc (Overfitting)', fontsize=12)
ax.set_title('Decision Tree: Overfitting vs Capacity', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xticks(depths)

ax = axes[1, 1]
gaps_rf = [train - val for train, val in zip([r['train_acc'] for r in rf_results], 
                                               [r['val_acc'] for r in rf_results])]
ax.plot(n_ests, gaps_rf, 's-', linewidth=2.5, markersize=10, color='purple', label='Random Forest')
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.fill_between([r['capacity'] for r in rf_results], 0, gaps_rf, alpha=0.3, color='purple')
ax.set_xlabel('Number of Trees', fontsize=12)
ax.set_ylabel('Train Acc - Val Acc (Overfitting)', fontsize=12)
ax.set_title('Random Forest: Overfitting vs Capacity', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

ax = axes[1, 2]
gaps_gb = [train - val for train, val in zip([r['train_acc'] for r in gb_results], 
                                               [r['val_acc'] for r in gb_results])]
ax.plot([r['capacity'] for r in gb_results], gaps_gb, '^-', linewidth=2.5, markersize=10, 
        color='darkgreen', label='Gradient Boosting')
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.fill_between([r['capacity'] for r in gb_results], 0, gaps_gb, alpha=0.3, color='darkgreen')
ax.set_xlabel('Number of Boosting Rounds', fontsize=12)
ax.set_ylabel('Train Acc - Val Acc (Overfitting)', fontsize=12)
ax.set_title('Gradient Boosting: Overfitting vs Capacity', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

plt.suptitle('Tree-Based Models: Capacity Scaling Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nKey Observations for Trees:")
print("1. Decision Trees show SHARP transition from underfitting to overfitting")
print("2. Random Forest smooths this transition through ensemble averaging")
print("3. Gradient Boosting shows gradual improvement with diminishing returns")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Performance vs Parameters
ax = axes[0, 0]
n_params = [r['n_params'] for r in transformer_results]
final_train = [r['final_train_acc'] for r in transformer_results]
final_val = [r['final_val_acc'] for r in transformer_results]
test_acc = [r['test_acc'] for r in transformer_results]

ax.plot(n_params, final_train, 'o-', linewidth=2, markersize=10, label='Train', color='blue')
ax.plot(n_params, final_val, 's-', linewidth=2, markersize=10, label='Validation', color='orange')
ax.plot(n_params, test_acc, '^-', linewidth=2, markersize=10, label='Test', color='green')
ax.set_xlabel('Number of Parameters', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Transformer: Model Size vs Performance', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

# Add annotations for each model
for i, r in enumerate(transformer_results):
    ax.annotate(r['name'].split('(')[0].strip(), 
                xy=(r['n_params'], r['final_val_acc']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=8, alpha=0.7)

# Overfitting gap
ax = axes[0, 1]
gaps = [train - val for train, val in zip(final_train, final_val)]
colors = plt.cm.viridis(np.linspace(0, 1, len(transformer_results)))

bars = ax.bar(range(len(transformer_results)), gaps, color=colors, alpha=0.7, edgecolor='black')
ax.set_xticks(range(len(transformer_results)))
ax.set_xticklabels([r['name'].split('(')[0].strip() for r in transformer_results], 
                    rotation=45, ha='right', fontsize=10)
ax.set_ylabel('Train Acc - Val Acc (Overfitting)', fontsize=12)
ax.set_title('Transformer: Overfitting vs Model Configuration', fontsize=13, fontweight='bold')
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.grid(True, alpha=0.3, axis='y')

# Training curves for different capacities
ax = axes[1, 0]
for i, r in enumerate(transformer_results):
    if i in [0, 2, 5]:  # Select small, medium, very large
        ax.plot(r['val_losses'], linewidth=2, label=r['name'], alpha=0.8)

ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Validation Loss', fontsize=12)
ax.set_title('Transformer: Training Dynamics by Capacity', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Final comparison
ax = axes[1, 1]
x_pos = np.arange(len(transformer_results))
width = 0.35

final_train_accs = [r['final_train_acc'] for r in transformer_results]
final_val_accs = [r['final_val_acc'] for r in transformer_results]

bars1 = ax.bar(x_pos - width/2, final_train_accs, width, label='Train', 
               color='blue', alpha=0.7, edgecolor='black')
bars2 = ax.bar(x_pos + width/2, final_val_accs, width, label='Validation', 
               color='orange', alpha=0.7, edgecolor='black')

ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Transformer: Train vs Validation Accuracy', fontsize=13, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels([r['name'].split('(')[0].strip() for r in transformer_results], 
                    rotation=45, ha='right', fontsize=10)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Transformer Models: Capacity Scaling Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nKey Observations for Transformers:")
print("1. More GRADUAL scaling with capacity")
print("2. Larger models can generalize well despite overparameterization")
print("3. Training dynamics show smooth convergence")

## Part 5: Direct Comparison: Trees vs Transformers

Let's directly compare how these two model families respond to increasing capacity.

### Key Metrics:
1. **Capacity** (normalized for comparison)
2. **Bias**: Distance from optimal performance
3. **Variance**: Generalization gap (train - val accuracy)
4. **Scaling behavior**: How quickly performance improves with capacity

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Normalize capacity for comparison
# For trees: use depth or number of estimators
# For transformers: use number of parameters

# Plot 1: Bias (measured as 1 - validation accuracy)
ax = axes[0, 0]

# Decision Trees
dt_depths = [r['depth'] for r in dt_results]
dt_bias = [1 - r['val_acc'] for r in dt_results]
ax.plot(dt_depths, dt_bias, 'o-', linewidth=2.5, markersize=10, 
        label='Decision Tree', color='brown', alpha=0.8)

# Random Forest (normalize n_estimators to similar scale)
rf_n_est = [r['capacity'] for r in rf_results]
rf_bias = [1 - r['val_acc'] for r in rf_results]
rf_normalized = [x / 10 for x in rf_n_est]  # Normalize for comparison
ax.plot(rf_normalized, rf_bias, 's-', linewidth=2.5, markersize=10, 
        label='Random Forest', color='green', alpha=0.8)

# Transformers (normalize params to similar scale)
trans_params = [r['n_params'] / 1000 for r in transformer_results]  # Scale to thousands
trans_bias = [1 - r['final_val_acc'] for r in transformer_results]
ax.plot(trans_params, trans_bias, '^-', linewidth=2.5, markersize=10, 
        label='Transformer', color='purple', alpha=0.8)

ax.set_xlabel('Relative Capacity (normalized)', fontsize=12)
ax.set_ylabel('Bias (1 - Validation Accuracy)', fontsize=12)
ax.set_title('Bias Reduction with Increasing Capacity', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

# Plot 2: Variance (generalization gap)
ax = axes[0, 1]

dt_variance = [r['train_acc'] - r['val_acc'] for r in dt_results]
ax.plot(dt_depths, dt_variance, 'o-', linewidth=2.5, markersize=10, 
        label='Decision Tree', color='brown', alpha=0.8)

rf_variance = [r['train_acc'] - r['val_acc'] for r in rf_results]
ax.plot(rf_normalized, rf_variance, 's-', linewidth=2.5, markersize=10, 
        label='Random Forest', color='green', alpha=0.8)

trans_variance = [r['final_train_acc'] - r['final_val_acc'] for r in transformer_results]
ax.plot(trans_params, trans_variance, '^-', linewidth=2.5, markersize=10, 
        label='Transformer', color='purple', alpha=0.8)

ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Relative Capacity (normalized)', fontsize=12)
ax.set_ylabel('Variance (Train - Val Accuracy)', fontsize=12)
ax.set_title('Variance Increase with Capacity', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Shade "danger zone" for high variance
ax.axhspan(0.1, 0.5, alpha=0.1, color='red', label='High Variance Zone')

# Plot 3: Bias-Variance Tradeoff Space
ax = axes[1, 0]

# Decision Tree trajectory
ax.plot(dt_bias, dt_variance, 'o-', linewidth=2.5, markersize=10, 
        label='Decision Tree', color='brown', alpha=0.8)
for i, depth in enumerate(dt_depths[::2]):  # Annotate every other point
    if i * 2 < len(dt_depths):
        ax.annotate(f'd={dt_depths[i*2]}', 
                    xy=(dt_bias[i*2], dt_variance[i*2]),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=8, alpha=0.7)

# Random Forest trajectory
ax.plot(rf_bias, rf_variance, 's-', linewidth=2.5, markersize=10, 
        label='Random Forest', color='green', alpha=0.8)

# Transformer trajectory
ax.plot(trans_bias, trans_variance, '^-', linewidth=2.5, markersize=10, 
        label='Transformer', color='purple', alpha=0.8)

# Mark optimal region
optimal_region = plt.Circle((0.15, 0.02), 0.05, color='gold', alpha=0.2)
ax.add_patch(optimal_region)
ax.text(0.15, 0.02, 'Optimal\nRegion', ha='center', va='center', 
        fontsize=10, fontweight='bold')

ax.set_xlabel('Bias (1 - Val Accuracy)', fontsize=12)
ax.set_ylabel('Variance (Train - Val Accuracy)', fontsize=12)
ax.set_title('Bias-Variance Tradeoff Space', fontsize=13, fontweight='bold')
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, alpha=0.3)

# Add arrows showing trajectory direction
ax.annotate('', xy=(dt_bias[-1], dt_variance[-1]), 
            xytext=(dt_bias[0], dt_variance[0]),
            arrowprops=dict(arrowstyle='->', lw=1.5, color='brown', alpha=0.3))
ax.annotate('', xy=(trans_bias[-1], trans_variance[-1]), 
            xytext=(trans_bias[0], trans_variance[0]),
            arrowprops=dict(arrowstyle='->', lw=1.5, color='purple', alpha=0.3))

# Plot 4: Scaling Efficiency
ax = axes[1, 1]

# How much does validation accuracy improve per unit of capacity?
dt_capacity_normalized = np.array(dt_depths) / max(dt_depths)
dt_val_accs = [r['val_acc'] for r in dt_results]

rf_capacity_normalized = np.array(rf_n_est) / max(rf_n_est)
rf_val_accs = [r['val_acc'] for r in rf_results]

trans_capacity_normalized = np.array([r['n_params'] for r in transformer_results]) / max([r['n_params'] for r in transformer_results])
trans_val_accs = [r['final_val_acc'] for r in transformer_results]

ax.plot(dt_capacity_normalized, dt_val_accs, 'o-', linewidth=2.5, markersize=10, 
        label='Decision Tree', color='brown', alpha=0.8)
ax.plot(rf_capacity_normalized, rf_val_accs, 's-', linewidth=2.5, markersize=10, 
        label='Random Forest', color='green', alpha=0.8)
ax.plot(trans_capacity_normalized, trans_val_accs, '^-', linewidth=2.5, markersize=10, 
        label='Transformer', color='purple', alpha=0.8)

ax.set_xlabel('Normalized Capacity (0-1)', fontsize=12)
ax.set_ylabel('Validation Accuracy', fontsize=12)
ax.set_title('Scaling Efficiency: Performance vs Capacity', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Add reference lines
ax.axhline(max(dt_val_accs + rf_val_accs + trans_val_accs), 
           color='gold', linestyle='--', alpha=0.5, linewidth=2, label='Best Performance')

plt.suptitle('Direct Comparison: Trees vs Transformers', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Part 6: Empirical Bias-Variance Decomposition

To truly measure bias and variance, we need to:
1. Train multiple models on different bootstrap samples
2. Calculate prediction variance across models (variance)
3. Calculate average prediction error (bias²)

This is the classic **bias-variance decomposition** experiment.

In [ ]:
def empirical_bias_variance_decomposition(model_class, model_params, X_train, y_train, 
                                          X_test, y_test, n_bootstrap=20):
    """
    Perform empirical bias-variance decomposition.
    Train multiple models on bootstrap samples and measure bias/variance.
    """
    n_samples = len(X_train)
    predictions = []
    
    print(f"Training {n_bootstrap} models on bootstrap samples...")
    for i in range(n_bootstrap):
        # Bootstrap sample
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot = X_train[indices]
        y_boot = y_train[indices]
        
        # Train model
        if 'transformer' in str(model_class).lower():
            # For transformer, need special handling
            # Skip for now or implement if needed
            continue
        else:
            model = model_class(**model_params)
            model.fit(X_boot, y_boot)
        
        # Predict on test set
        if hasattr(model, 'predict_proba'):
            pred = model.predict_proba(X_test)
        else:
            pred = model.predict(X_test)
        predictions.append(pred)
    
    predictions = np.array(predictions)  # shape: (n_bootstrap, n_test, n_classes)
    
    # Calculate metrics
    # Average prediction across all models
    if len(predictions.shape) == 3:  # Probabilities
        mean_predictions = np.mean(predictions, axis=0)
        # Variance: variance of predictions across models
        variance = np.mean(np.var(predictions, axis=0))
    else:  # Hard predictions
        mean_predictions = np.mean(predictions, axis=0)
        variance = np.mean(np.var(predictions, axis=0))
    
    # Bias: error of average prediction
    # Convert to class predictions
    if len(predictions.shape) == 3:
        avg_pred_class = np.argmax(mean_predictions, axis=1)
    else:
        avg_pred_class = np.round(mean_predictions).astype(int)
    
    bias_squared = np.mean(avg_pred_class != y_test)
    
    return bias_squared, variance, predictions

# Compare Decision Tree at different depths
print("="*70)
print("EMPIRICAL BIAS-VARIANCE DECOMPOSITION")
print("="*70)

depths_to_test = [3, 10, None]
dt_bias_variance = []

for depth in depths_to_test:
    print(f"\nDecision Tree (depth={depth}):")
    bias_sq, variance, _ = empirical_bias_variance_decomposition(
        DecisionTreeClassifier,
        {'max_depth': depth, 'random_state': 42},
        X_train, y_train, X_test, y_test,
        n_bootstrap=20
    )
    dt_bias_variance.append({
        'depth': depth if depth else 'unlimited',
        'bias_squared': bias_sq,
        'variance': variance
    })
    print(f"  Bias² (error rate): {bias_sq:.4f}")
    print(f"  Variance: {variance:.4f}")

# Compare Random Forest with different n_estimators
rf_n_ests = [10, 50, 200]
rf_bias_variance = []

for n_est in rf_n_ests:
    print(f"\nRandom Forest (n_estimators={n_est}):")
    bias_sq, variance, _ = empirical_bias_variance_decomposition(
        RandomForestClassifier,
        {'n_estimators': n_est, 'max_depth': 10, 'random_state': 42, 'n_jobs': -1},
        X_train, y_train, X_test, y_test,
        n_bootstrap=20
    )
    rf_bias_variance.append({
        'n_estimators': n_est,
        'bias_squared': bias_sq,
        'variance': variance
    })
    print(f"  Bias² (error rate): {bias_sq:.4f}")
    print(f"  Variance: {variance:.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Decision Tree
ax = axes[0]
depths_plot = [str(d['depth']) for d in dt_bias_variance]
bias_vals = [d['bias_squared'] for d in dt_bias_variance]
var_vals = [d['variance'] for d in dt_bias_variance]

x_pos = np.arange(len(depths_plot))
width = 0.35

bars1 = ax.bar(x_pos - width/2, bias_vals, width, label='Bias²', 
               color='steelblue', alpha=0.8, edgecolor='black')
bars2 = ax.bar(x_pos + width/2, var_vals, width, label='Variance', 
               color='coral', alpha=0.8, edgecolor='black')

ax.set_ylabel('Error Magnitude', fontsize=12)
ax.set_xlabel('Max Depth', fontsize=12)
ax.set_title('Decision Tree: Empirical Bias-Variance Decomposition', 
             fontsize=13, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(depths_plot)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

# Random Forest
ax = axes[1]
n_ests_plot = [str(d['n_estimators']) for d in rf_bias_variance]
bias_vals = [d['bias_squared'] for d in rf_bias_variance]
var_vals = [d['variance'] for d in rf_bias_variance]

x_pos = np.arange(len(n_ests_plot))

bars1 = ax.bar(x_pos - width/2, bias_vals, width, label='Bias²', 
               color='steelblue', alpha=0.8, edgecolor='black')
bars2 = ax.bar(x_pos + width/2, var_vals, width, label='Variance', 
               color='coral', alpha=0.8, edgecolor='black')

ax.set_ylabel('Error Magnitude', fontsize=12)
ax.set_xlabel('Number of Trees', fontsize=12)
ax.set_title('Random Forest: Empirical Bias-Variance Decomposition', 
             fontsize=13, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(n_ests_plot)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## Part 7: Explaining the Fundamental Differences

### Why do Transformers and Trees respond differently to capacity increases?

---

## 1. **Learning Mechanism**

### Trees (Greedy, Local):
- **Greedy splitting**: Each split optimizes locally, not globally
- **No gradient flow**: Can't backpropagate to fix earlier mistakes
- **Hard partitions**: Data points are definitively assigned to regions
- **Result**: Need exponentially more capacity (depth) to model interactions

### Transformers (Gradient-based, Global):
- **End-to-end optimization**: Gradients flow through entire network
- **Soft attention**: Probabilistic combination of all features
- **Continuous representations**: Smooth, differentiable functions
- **Result**: Can efficiently model complex interactions with modest capacity

---

## 2. **Capacity Scaling**

### Trees:
```
Capacity ≈ 2^depth  (exponential)
```
- Doubling depth quadruples capacity
- Each level doubles possible regions
- **Sharp transition**: Small depth change = huge capacity change

### Transformers:
```
Capacity ≈ layers × d_model²  (polynomial)
```
- Linear scaling with depth
- Quadratic scaling with width
- **Gradual transition**: Small changes = small capacity changes

---

## 3. **Implicit Regularization**

### Trees:
- **No implicit regularization** (except ensemble averaging)
- Need explicit constraints (max_depth, min_samples_split)
- Overfitting happens suddenly when constraints are relaxed

### Transformers:
- **SGD implicit bias**: Gradient descent finds flat minima
- **Overparameterization helps**: More parameters → better optimization landscape
- **Early stopping**: Training dynamics naturally regulate
- Can benefit from models larger than strictly necessary

---

## 4. **Feature Interactions**

### Trees:
- **Hierarchical splits**: Feature A at level 1, Feature B at level 2
- **Order matters**: Different orderings create different models
- **Limited interaction**: Depth d → can model d-way interactions
- **Example**: To learn X₁ × X₂ × X₃, need depth ≥ 3

### Transformers:
- **Self-attention**: All features interact simultaneously
- **Order-invariant**: (with positional encoding, order is added separately)
- **Rich interactions**: Single layer can model complex feature combinations
- **Example**: Can learn X₁ × X₂ × X₃ in one attention layer

---

## 5. **Variance Sources**

### Trees - High Variance from:
1. **Bootstrap sensitivity**: Different samples → different split points
2. **High sensitivity**: Small data change → different tree structure
3. **Discrete decisions**: Binary splits are unstable
4. **Mitigation**: Ensemble methods (bagging, boosting)

### Transformers - Lower Variance from:
1. **Continuous optimization**: Smooth loss landscape
2. **Regularization**: Dropout, weight decay, gradient noise
3. **Soft decisions**: Attention weights are continuous
4. **Parameter sharing**: Same weights used everywhere

---

## 6. **The Capacity Paradox**

### Trees:
- ❌ **More capacity** (depth) → immediate overfitting
- ❌ **More trees** → diminishing returns past ~100-200 trees
- ✅ **Sweet spot**: Moderate depth (5-10) with ensemble

### Transformers:
- ✅ **More capacity** → often better, even if "overparameterized"
- ✅ **More layers** → can still generalize with proper training
- ✅ **Modern regime**: Bigger models trained with good practices generalize better

This is the **double descent** phenomenon: 
- Classical regime: more capacity → overfitting
- Modern regime: even more capacity → good generalization again

---

## Summary Table

| Aspect | Trees | Transformers |
|--------|-------|--------------|
| **Capacity Scaling** | Exponential (2^depth) | Polynomial (layers × d²) |
| **Learning** | Greedy, local | Gradient-based, global |
| **Interactions** | Hierarchical, limited | Simultaneous, rich |
| **Variance** | High (discrete splits) | Lower (continuous) |
| **Regularization** | Explicit only | Implicit + explicit |
| **Transition** | Sharp (cliff) | Gradual (slope) |
| **Overparameterization** | Always bad | Can be good |

In [ ]:
# Create comprehensive summary figure
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)

# Row 1: Capacity curves comparison
ax1 = fig.add_subplot(gs[0, :])

# Normalize all capacities to 0-1 range for comparison
dt_cap_norm = np.array([r['depth'] for r in dt_results]) / max([r['depth'] for r in dt_results])
dt_val = [r['val_acc'] for r in dt_results]
dt_train = [r['train_acc'] for r in dt_results]

rf_cap_norm = np.array([r['capacity'] for r in rf_results]) / max([r['capacity'] for r in rf_results])
rf_val = [r['val_acc'] for r in rf_results]
rf_train = [r['train_acc'] for r in rf_results]

trans_cap_norm = np.array([r['n_params'] for r in transformer_results]) / max([r['n_params'] for r in transformer_results])
trans_val = [r['final_val_acc'] for r in transformer_results]
trans_train = [r['final_train_acc'] for r in transformer_results]

# Plot training curves
ax1.plot(dt_cap_norm, dt_train, 'o--', linewidth=2, markersize=8, 
         label='Decision Tree (Train)', color='brown', alpha=0.5)
ax1.plot(dt_cap_norm, dt_val, 'o-', linewidth=2.5, markersize=10, 
         label='Decision Tree (Val)', color='brown', alpha=0.9)

ax1.plot(rf_cap_norm, rf_train, 's--', linewidth=2, markersize=8, 
         label='Random Forest (Train)', color='green', alpha=0.5)
ax1.plot(rf_cap_norm, rf_val, 's-', linewidth=2.5, markersize=10, 
         label='Random Forest (Val)', color='green', alpha=0.9)

ax1.plot(trans_cap_norm, trans_train, '^--', linewidth=2, markersize=8, 
         label='Transformer (Train)', color='purple', alpha=0.5)
ax1.plot(trans_cap_norm, trans_val, '^-', linewidth=2.5, markersize=10, 
         label='Transformer (Val)', color='purple', alpha=0.9)

ax1.set_xlabel('Normalized Capacity (0 = minimum, 1 = maximum)', fontsize=13, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=13, fontweight='bold')
ax1.set_title('Model Family Comparison: How Performance Scales with Capacity', 
              fontsize=14, fontweight='bold')
ax1.legend(fontsize=10, ncol=3, loc='lower right')
ax1.grid(True, alpha=0.3)

# Add annotations
ax1.annotate('Sharp\ntransition', xy=(0.3, 0.8), xytext=(0.3, 0.65),
            arrowprops=dict(arrowstyle='->', lw=2, color='brown'),
            fontsize=11, ha='center', color='brown', fontweight='bold')

ax1.annotate('Gradual\nimprovement', xy=(0.7, 0.72), xytext=(0.7, 0.60),
            arrowprops=dict(arrowstyle='->', lw=2, color='purple'),
            fontsize=11, ha='center', color='purple', fontweight='bold')

# Row 2: Overfitting patterns
ax2 = fig.add_subplot(gs[1, 0])
dt_gaps = [r['train_acc'] - r['val_acc'] for r in dt_results]
ax2.fill_between(dt_cap_norm, 0, dt_gaps, alpha=0.5, color='brown')
ax2.plot(dt_cap_norm, dt_gaps, 'o-', linewidth=2.5, markersize=10, color='brown')
ax2.set_title('Decision Tree\nOverfitting Pattern', fontsize=12, fontweight='bold')
ax2.set_xlabel('Normalized Capacity', fontsize=11)
ax2.set_ylabel('Train - Val Accuracy', fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
ax2.text(0.5, max(dt_gaps)*0.8, 'Sharp\nincrease', fontsize=10, ha='center',
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

ax3 = fig.add_subplot(gs[1, 1])
rf_gaps = [r['train_acc'] - r['val_acc'] for r in rf_results]
ax3.fill_between(rf_cap_norm, 0, rf_gaps, alpha=0.5, color='green')
ax3.plot(rf_cap_norm, rf_gaps, 's-', linewidth=2.5, markersize=10, color='green')
ax3.set_title('Random Forest\nOverfitting Pattern', fontsize=12, fontweight='bold')
ax3.set_xlabel('Normalized Capacity', fontsize=11)
ax3.set_ylabel('Train - Val Accuracy', fontsize=11)
ax3.grid(True, alpha=0.3)
ax3.axhline(0, color='black', linestyle='--', alpha=0.5)
ax3.text(0.5, max(rf_gaps)*0.8, 'Controlled\nby ensemble', fontsize=10, ha='center',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

ax4 = fig.add_subplot(gs[1, 2])
trans_gaps = [r['final_train_acc'] - r['final_val_acc'] for r in transformer_results]
ax4.fill_between(trans_cap_norm, 0, trans_gaps, alpha=0.5, color='purple')
ax4.plot(trans_cap_norm, trans_gaps, '^-', linewidth=2.5, markersize=10, color='purple')
ax4.set_title('Transformer\nOverfitting Pattern', fontsize=12, fontweight='bold')
ax4.set_xlabel('Normalized Capacity', fontsize=11)
ax4.set_ylabel('Train - Val Accuracy', fontsize=11)
ax4.grid(True, alpha=0.3)
ax4.axhline(0, color='black', linestyle='--', alpha=0.5)
ax4.text(0.5, max(trans_gaps)*0.8, 'Minimal\noverfitting', fontsize=10, ha='center',
         bbox=dict(boxstyle='round', facecolor='lavender', alpha=0.7))

# Row 3: Summary statistics
ax5 = fig.add_subplot(gs[2, :])

# Create summary table data
model_types = ['Decision Tree\n(max capacity)', 'Random Forest\n(max capacity)', 
               'Transformer\n(max capacity)']

best_val_accs = [
    max([r['val_acc'] for r in dt_results]),
    max([r['val_acc'] for r in rf_results]),
    max([r['final_val_acc'] for r in transformer_results])
]

min_overfit_gaps = [
    min([abs(r['train_acc'] - r['val_acc']) for r in dt_results]),
    min([abs(r['train_acc'] - r['val_acc']) for r in rf_results]),
    min([abs(r['final_train_acc'] - r['final_val_acc']) for r in transformer_results])
]

max_overfit_gaps = [
    max([abs(r['train_acc'] - r['val_acc']) for r in dt_results]),
    max([abs(r['train_acc'] - r['val_acc']) for r in rf_results]),
    max([abs(r['final_train_acc'] - r['final_val_acc']) for r in transformer_results])
]

x = np.arange(len(model_types))
width = 0.25

bars1 = ax5.bar(x - width, best_val_accs, width, label='Best Val Accuracy', 
                color='gold', alpha=0.8, edgecolor='black', linewidth=2)
bars2 = ax5.bar(x, min_overfit_gaps, width, label='Min Overfitting Gap', 
                color='lightgreen', alpha=0.8, edgecolor='black', linewidth=2)
bars3 = ax5.bar(x + width, max_overfit_gaps, width, label='Max Overfitting Gap', 
                color='salmon', alpha=0.8, edgecolor='black', linewidth=2)

ax5.set_ylabel('Value', fontsize=13, fontweight='bold')
ax5.set_title('Summary: Peak Performance and Overfitting Range', fontsize=14, fontweight='bold')
ax5.set_xticks(x)
ax5.set_xticklabels(model_types, fontsize=11)
ax5.legend(fontsize=11, loc='upper left')
ax5.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.suptitle('COMPREHENSIVE COMPARISON: TREES vs TRANSFORMERS\nCapacity Scaling and Bias-Variance Tradeoff', 
             fontsize=16, fontweight='bold')
plt.show()

# Print final summary
print("\n" + "="*80)
print(" "*25 + "FINAL SUMMARY")
print("="*80)
print("\nBest Validation Accuracy:")
print(f"  Decision Tree: {max([r['val_acc'] for r in dt_results]):.4f}")
print(f"  Random Forest: {max([r['val_acc'] for r in rf_results]):.4f}")
print(f"  Transformer:   {max([r['final_val_acc'] for r in transformer_results]):.4f}")

print("\nOverfitting Behavior:")
print(f"  Decision Tree: Overfitting gap ranges from {min_overfit_gaps[0]:.4f} to {max_overfit_gaps[0]:.4f}")
print(f"  Random Forest: Overfitting gap ranges from {min_overfit_gaps[1]:.4f} to {max_overfit_gaps[1]:.4f}")
print(f"  Transformer:   Overfitting gap ranges from {min_overfit_gaps[2]:.4f} to {max_overfit_gaps[2]:.4f}")

print("\nKey Insight:")
print("  • Trees show SHARP transition: low capacity (high bias) → high capacity (high variance)")
print("  • Transformers show GRADUAL scaling: can handle high capacity without severe overfitting")
print("  • Ensembles (RF) smooth the transition but still show the sharp pattern")
print("="*80)

## Conclusions: Why the Tradeoff is Different

### Summary of Findings:

1. **Capacity Scaling**
   - **Trees**: Exponential capacity growth (2^depth) leads to sharp overfitting
   - **Transformers**: Polynomial capacity growth allows gradual scaling
   
2. **Learning Dynamics**
   - **Trees**: Greedy, local decisions without global optimization
   - **Transformers**: End-to-end gradient optimization with implicit regularization

3. **Bias-Variance Behavior**
   - **Trees**: Classic U-shaped curve - clear sweet spot
   - **Transformers**: More forgiving - larger models can still generalize

4. **Feature Interactions**
   - **Trees**: Limited by depth, hierarchical interactions
   - **Transformers**: Rich interactions through attention, all features interact

---

## Practical Implications:

### When to Use Trees:
✅ **Tabular data with moderate complexity**
✅ **Interpretability is crucial**
✅ **Limited computational resources**
✅ **Small to medium datasets**
✅ **Feature importance analysis needed**

**Best Practice**: Use ensemble methods (Random Forest, GradientBoosting) with moderate depth

### When to Use Transformers:
✅ **Complex feature interactions**
✅ **Large datasets available**
✅ **Computational resources available**
✅ **Sequential or relational data**
✅ **Transfer learning beneficial**

**Best Practice**: Start with moderate size, scale up if needed, use proper regularization

---

## The Modern Paradigm Shift:

**Classical ML (Trees)**:
```
More parameters → Overfitting (always bad)
Sweet spot at moderate complexity
```

**Modern Deep Learning (Transformers)**:
```
More parameters → Better optimization → Implicit regularization
Can benefit from overparameterization
Double descent phenomenon
```

---

## Key Takeaway:

The bias-variance tradeoff manifests differently because:
- **Architecture**: How models represent functions
- **Optimization**: How models learn from data
- **Capacity scaling**: How complexity grows with parameters
- **Regularization**: How models prevent overfitting

Understanding these differences helps you:
1. Choose the right model family for your problem
2. Set appropriate hyperparameters
3. Diagnose and fix performance issues
4. Know when to scale up vs regularize